# Урок 4. Изучение Spark Structure Streaming
**Условие**: используйте источник rate, напишите код, который создаст дополнительный столбец, который будет выводить сумму только нечётных чисел.

*Не сразу поняла,что среда не выводит результат в консоль. Добавила чтение данных из памяти*

In [ ]:
#будет скрыта установка
!pip install pyspark >> None


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum

# Инициализация Spark сессии
spark = SparkSession.builder.appName("SumOddNumbers").getOrCreate()

# Чтение данных из источника "rate"
df = spark.readStream.format("rate").load()

# Фильтрация нечетных чисел
df_odd = df.filter(col("value") % 2 == 1)

# Подсчет суммы нечетных чисел
df_odd_sum = df_odd.groupBy().agg(spark_sum("value").alias("odd_sum"))

# Вывод результатов в консоль
query = df_odd_sum.writeStream.outputMode("update").format("console").start()

# Функция для остановки стриминга и завершения сессии
def stop_streaming():
    query.stop()
    spark.stop()

try:
    # Ожидание завершения записи (ограниченное по времени)
    query.awaitTermination(60)  # Ждём 60 секунд
except KeyboardInterrupt:
    print("Streaming interrupted. Stopping the query and Spark session.")
    stop_streaming()
except Exception as e:
    print(f"An error occurred: {e}")
    stop_streaming()
finally:
    stop_streaming()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, count
import time

# Создаем SparkSession
spark = SparkSession.builder.appName("SumOddNumbers").getOrCreate()

# Создаем поток данных с использованием источника `rate`
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# Оставляем только те строки, где значение нечетное
filtered_df = streaming_df.filter(col("value") % 2 == 1)

# Подсчет суммы нечетных значений и количества строк
sum_odds = filtered_df.agg(
    spark_sum("value").alias("odd_sum"),
    count("value").alias("odd_count")
)

# Записываем поток данных в память
query = sum_odds.writeStream.outputMode("complete").format("memory").queryName("oddSumStream").start()

# Ожидание завершения записи в течение 10 секунд
time.sleep(10)

# Останавливаем поток
query.stop()

# Чтение данных из памяти!!!
odd_sum_data = spark.sql("SELECT * FROM oddSumStream")
odd_sum_data.show()

# Остановить SparkSession
spark.stop()


+-------+---------+
|odd_sum|odd_count|
+-------+---------+
|   2025|       45|
+-------+---------+

